# Task 2

Concatenation of multiple measurement files of our lidar system Polly-XT and identification of 
calibration periods.

Wenfu Sun 2021-07-01

## 1. Load modules

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import glob

## 2. Data preparation and concatenation

In [2]:
# Get a sorted list of all .nc files
nc_files = sorted(glob.glob('Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS_*.nc'))
nc_files

['Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS_00_00_01.nc',
 'Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS_06_00_01.nc',
 'Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS_12_00_01.nc',
 'Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS_18_00_01.nc']

In [3]:
# Add all .nc files into an empty list and 
ds_list = []
for i_file in nc_files:
    ds_list.append(xr.open_dataset(i_file))
ds_list

[<xarray.Dataset>
 Dimensions:                        (channel: 13, coordinates: 2, date_time: 2, height: 6400, polynomial: 6, time: 720)
 Dimensions without coordinates: channel, coordinates, date_time, height, polynomial, time
 Data variables: (12/21)
     raw_signal                     (time, height, channel) int32 ...
     measurement_shots              (time, channel) int32 ...
     measurement_time               (time, date_time) int32 ...
     depol_cal_angle                (time) float32 ...
     measurement_height_resolution  float32 ...
     laser_rep_rate                 float32 ...
     ...                             ...
     deadtime_polynomial            (polynomial, channel) float32 ...
     deadtime_polynomial_error      (polynomial, channel) float32 ...
     discr_level                    (channel) float32 ...
     pm_voltage                     (channel) float32 ...
     pinhole                        float32 ...
     zenithangle                    float32 ...
 Attri

In [4]:
# Do the concatenation by the dimension of 'time'
ds = xr.concat(ds_list, dim='time')
ds.time

<xarray.DataArray 'time' (time: 2880)>
array([   0,    1,    2, ..., 2877, 2878, 2879])
Dimensions without coordinates: time

### The number of 'time' is 2880, which is the sum of the those number for 4 original NC files 

## 3. Values replacement
Here I create a copy of 'raw_signal' as 'processed_raw_signal' and process the later variable, which enables me to check if the replacement is done.

In [5]:
# Use the copy() function to create the 'processed_raw_signal' variable
ds['processed_raw_signal'] = ds['raw_signal'].copy()

# The NaN is shown as '-2147483648' when dtype is int32. Here I change the dtype from the int32 to float32, by which the 'nan' can be shown
ds['processed_raw_signal'] = ds['processed_raw_signal'].astype('float32')

# Replace the data in 'processed_raw_signal' for time periods where 'depol_cal_angle' is not 0 by NaN values
ds['processed_raw_signal'][ds['depol_cal_angle'] != 0, :, :] = np.nan

## 4. Check if the replacement is done
### Find non-zero value in 'depol_cal_angle'

In [6]:
DCA_non_zero = ds['depol_cal_angle'][ds['depol_cal_angle'] != 0]
DCA_non_zero

<xarray.DataArray 'depol_cal_angle' (time: 63)>
array([135.  , 135.  , 135.  , 135.  , 135.  , 135.  , 135.  , 135.  ,
       135.  , 135.  , 225.  , 225.  , 225.  , 225.  , 225.  , 225.  ,
       225.  , 225.  , 225.  , 225.  , 253.88, 135.  , 135.  , 135.  ,
       135.  , 135.  , 135.  , 135.  , 135.  , 135.  , 135.  , 225.  ,
       225.  , 225.  , 225.  , 225.  , 225.  , 225.  , 225.  , 225.  ,
       225.  , 252.76, 135.  , 135.  , 135.  , 135.  , 135.  , 135.  ,
       135.  , 135.  , 135.  , 135.  , 225.  , 225.  , 225.  , 225.  ,
       225.  , 225.  , 225.  , 225.  , 225.  , 225.  , 252.31],
      dtype=float32)
Dimensions without coordinates: time
Attributes:
    long_name:  Angle of Polarizer for Depol 
    units:      degree

### Find Indexes of non-zero value in 'depol_cal_angle' along the 'time' dimention

In [7]:
DCA_non_zero_indexs = ds['depol_cal_angle']['time'][ds['depol_cal_angle'] != 0]
DCA_non_zero_indexs

<xarray.DataArray 'time' (time: 63)>
array([ 301,  302,  303,  304,  305,  306,  307,  308,  309,  310,  311,  312,
        313,  314,  315,  316,  317,  318,  319,  320,  321, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2581, 2582, 2583, 2584, 2585, 2586,
       2587, 2588, 2589, 2590, 2591, 2592, 2593, 2594, 2595, 2596, 2597, 2598,
       2599, 2600, 2601])
Dimensions without coordinates: time

### Here is a demonstration when time is 302

In [8]:
ds['raw_signal'].sel(time=302).values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [9]:
ds['processed_raw_signal'].sel(time=302).values

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

### Now check if all the replacement is done

In [10]:
failed_count = 0
for i in DCA_non_zero_indexs.values:
    if not np.isnan(ds['processed_raw_signal'].sel(time=i).values).all():
        failed_count = failed_count + 1
        print(f'Time {i} fail')
if failed_count == 0:
    print(f'Successful')

Successful


## 5. Save the new NetCDF file

In [11]:
ds.to_netcdf(path='Data/02-Concatenation_of_measurement_files/2021_06_09_Wed_LACROS.nc')